# Introducción a la manipulación de datos con pandas - II parte

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://numfocus.org/wp-content/uploads/2016/07/pandas-logo-300.png" width="400px" height="400px" />

> La clase pasada comenzamos a ver las funcionalidades de la librería [pandas](https://pandas.pydata.org/). Básicamente, aprendimos a cargar datos desde archivos `.csv`, aprendimos a seleccionar subcojuntos de estos datos mediante indización (obtener ciertas filas, o ciertas columnas, o ciertas filas y columnas determinadas). Finalmente, aprendimos a filtrar datos, es decir, a seleccionar registros de la base de datos que satisfagan cierta condición.

> Hoy nos enfocaremos a ver como reunir varias bases de datos que poseen información complementaria relativa a un mismo problema, para obtener una sola tabla con la información condensada.

Referencias:
- https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
___

# 0. Motivación

En cualquier situación relacionada con ciencia de datos en la vida real, no pasarán más de 10 minutos sin que aparezca la necesidad de unir (`merge` o `join`) dos bases de datos complementarias para formar tu conjunto de datos para el análisis.

La unión de DataFrames es un proceso fundamental, que cualquier analista de datos en formación debe aprender. En esta clase aprenderemos cómo hacerlo, y en el camino responderemos las siguientes preguntas:

- ¿Qué son el `merge` o `join` de dos DataFrames?

- ¿Qué son `inner`, `outer`, `left` y `right` `joins`?



## Datos para la clase

En esta clase, seguiremos trabajando con los datos de la clase pasada:

En la carpeta "data" tenemos los archivos 

- "customers_data.csv": datos propios de los clientes de la empresa, 
- "sessions_data.csv": inicios de sesión de los clientes en la plataforma web de compras (similar a amazon),
- "transactions_data.csv": transacciones asociadas a cada inicio de sesión, y
- "products_data.csv": información de los productos. 

Podemos cargar cada uno de los archivos `.csv` como DataFrames de Pandas usando la función `read_csv()` de pandas, y examinar los contenidos de cada uno con el método `head()`.

In [ ]:
# Importar pandas


In [ ]:
# Cargar customers_data


In [ ]:
# Cargar sessions_data


In [ ]:
# Cargar transactions_data


In [ ]:
# Cargar products_data


Explorar los contenidos de cada tabla:

In [ ]:
# Explorar customers_data


In [ ]:
# Explorar sessions_data


In [ ]:
# Explorar transactions_data


In [ ]:
# Explorar products_data


## Bonus: formato de fechas y horas

Como vimos la vez pasada, es relevante en general un correcto manejo de las fechas y horas en las bases de datos.

Lo primero que se tiene que hacer para un correcto manejo de las fechas (y horas) es identificar las columnas o variables que contienen fechas. Por ejemplo, de la tabla `customers_data`:

Lo que sigue, es especificarle a pandas el formato en que vienen esas fechas con la función `to_datetime()` de pandas

In [ ]:
# Ayuda en la función to_datetime


In [ ]:
# Especificar el formato de fechas en la tabla customers_data


Hagan esto mismo para todas las tablas:

In [ ]:
# Especificar el formato de fechas en la tabla sessions_data


In [ ]:
# Especificar el formato de fechas en la tabla transactions_data


In [ ]:
# Especificar el formato de fechas en la tabla products_data


## Volviendo al problema ...

Hay columnas o variables que relacionan las tablas. Por ejemplo:

- Los clientes pueden iniciar sesión en la plataforma cuantas veces quieran. De manera que hay una relación uno a muchos entre las tablas "customers_data" y "sessions_data", mediante la variable "customer_id".

- Cuando se efectúa una transacción se supone que se está comprando un producto. Por lo tanto hay una relación uno a muchos entre las tablas "transactions_data"  y "products_data", mediante la variable "product_id".

## Problemas

Primero, quisieramos determinar cuál es el dispositivo preferido por zona.

Luego, quisiéramos determinar cuál es la marca preferida por zona.

Para resolver lo anterior necesitamos unir ("merge" o "join") nuestros datasets en uno solo para el análisis.

___
# 1. Uniendo dos DataFrames...

> “Merging” two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

Las palabras “merge” y “join” se usan indistintamente en Pandas, y en otros lenguajes como SQL y R. En Pandas, hay métodos separados “merge” y “join”, que realizan cosas similares (personalmente uso el método "merge"), y la función "merge".

Vamos a concentrarnos en el **primer problema**. En este escenario, necesitamos llevar a cabo los siguientes pasos:

- Para cada fila en el dataset `sessions_data`, debemos hacer una nueva columna que contenga el "zip_code" respectivo de cada cliente.
- Una vez hagamos esto, obtenemos la moda de los dispositivos para cada cliente.

**¿Podemos usar un ciclo?**

Claro que si. Se podría escribir un ciclo para esta tarea. Éste iría a través de cada fila en `sessions_data`, y a cada "user_id" asignar el valor de la nueva columna con la zona respectiva.

Sin embargo, usar ciclos haría nuestra tarea mucho más lenta y plagada de más código que el necesario que si se usara la función (método) `join()`.

De forma que, para estas situaciones, **nunca usar ciclos**.

Para ver cómo podemos hacer lo anterior, veamos la ayuda de la función `merge()` de pandas:

In [ ]:
# Ayuda de merge()


Ahora, veamos como podemos añadir el código postal a la tabla `sessions_data`, usando la función `merge()` de pandas:

In [ ]:
# Uso de la función merge()


La función `merge()` es el objetivo principal de esta clase. Básicamente, la operación de unir dos DataFrames hace lo siguiente: 

- toma el DataFrame de la izquierda (argumento left=), 
- el DataFrame de la derecha (argumento right=),
- la columna donde se va a unir (argumento on=), y
- la forma en que se va a unir (argumento how=).

La variable en común sobre la que se hace la unión está especificada en el argumento `on`.

Con este resultado, podemos filtrar por zona y luego obtener la moda.

In [ ]:
# Obtener la moda por zona


### ¿Qué son los tipos de unión inner, left, right y outer?

En nuestro ejemplo, unimos `sessions_data` con `customers_data` sobre la columna "custumers_id". En este caso, en ambas tablas existían todos los posibles valores de "customers_id" (1, 2, 3, 4, 5).

¿Qué pasa si esta situación no se cumple?

Por ejemplo, realicemos los siguientes cambios artificiales:

In [ ]:
# artificial sessions_data
art_sessions = sessions[sessions['customer_id'] != 2]
# artificial customers_data
art_customers = customers[customers['customer_id'] != 5]

Por defecto, la operación `merge()` de pandas actpua con un merge tipo "inner". Un "inner merge", guarda únicamente los valores comúnes (en la columna especificada en el argumento `on=`) de ambos DataFrames.

Por ejemplo:

In [ ]:
# inner join


Los otros comportamientos se pueden revisar mejor a través de ejemplos:

In [ ]:
# left join


In [ ]:
# right join


In [ ]:
# outer join


Estos comportamientos son muy intuitivos a partir de sus valores.

Así mismo, sus usos son muy intuitivos (lo sabrán cuando se enfrenten a problemas reales).

### Resolvamos el problema 2

Determinar cuál es la marca preferida por zona.

Uso de los métodos `groupby()` y `agg()`:

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>